# Python code for database

In [2]:
import sqlite3

In [3]:
airline2 = sqlite3.connect('airline2.db')
cursor = airline2.cursor()
print("Database created/connected successfully.")

Database created/connected successfully.


In [10]:
import pandas as pd
import glob
import csv

To create tables

In [ ]:
airports = pd.read_csv("airports.csv")
carriers = pd.read_csv("carriers.csv")
planes = pd.read_csv("plane-data.csv")

ontime = pd.concat(map(pd.read_csv, glob.glob('200*.csv')))

In [ ]:
airports.to_sql('airports', con= airline2, index = False)
carriers.to_sql('carriers', con= airline2, index = False)
planes.to_sql('planes', con= airline2, index = False)

ontime.to_sql('ontime', con=airline2, index = False)

Querying

In [22]:
q1 = cursor.execute('''SELECT planes.model, SUM(1) As numplanes, AVG(DepDelay) As AverDelay, planes.manufacturer 
               FROM ontime LEFT JOIN planes ON ontime.tailnum=planes.tailnum 
               WHERE (DepDelay>0 AND Diverted = 0)
               GROUP BY planes.model
               ORDER BY AverDelay ASC
               LIMIT 0,1;''').fetchall()

q1=pd.DataFrame(q1)

In [23]:
q1.to_csv('airline2q1.csv',index=False)

In [25]:
q2 = cursor.execute('''SELECT ontime.Dest, SUM(1) As inflights
FROM ontime LEFT JOIN airports ON ontime.Dest=airports.iata
WHERE cancelled=0
GROUP BY ontime.Dest
ORDER BY inflights DESC
LIMIT 0,1;''').fetchall()

q2=pd.DataFrame(q2)

In [26]:
q2.to_csv('airline2q2.csv',index=False)

In [27]:
q3 = cursor.execute('''SELECT ontime.UniqueCarrier, SUM(1) As numflights, carriers.Description
FROM ontime LEFT JOIN carriers ON ontime.UniqueCarrier=carriers.Code
WHERE cancelled>0
GROUP BY ontime.UniqueCarrier
ORDER BY numflights DESC
LIMIT 0,1;''').fetchall()

q3=pd.DataFrame(q3)

In [28]:
q3.to_csv('airline2q3.csv',index=False)

In [29]:
q4 = cursor.execute('''WITH t1 AS (SELECT ontime.UniqueCarrier, SUM(1) As numflights, carriers.Description
FROM ontime LEFT JOIN carriers ON ontime.UniqueCarrier=carriers.Code
WHERE cancelled>0
GROUP BY ontime.UniqueCarrier
ORDER BY numflights DESC),
t2 AS (SELECT ontime.UniqueCarrier, SUM(1) As numflights, carriers.Description
FROM ontime LEFT JOIN carriers ON ontime.UniqueCarrier=carriers.Code
GROUP BY ontime.UniqueCarrier
ORDER BY numflights DESC)

SELECT t2.UniqueCarrier, (CAST(t1.numflights AS REAL)/CAST(t2.numflights AS REAL)) AS Ratio 
FROM t2 
JOIN t1 ON t1.UniqueCarrier=t2.UniqueCarrier
ORDER BY Ratio DESC''').fetchall()

q4=pd.DataFrame(q4)

In [30]:
q4.to_csv('airline2q4.csv',index=False)

In [31]:
airline2.close()